## Installing the git LFS fylesystem support

In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs -y
!git lfs install

Detected operating system as amzn/2.
Checking for curl...
Detected curl...
done.
Installing pygpgme to verify GPG signatures...
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
github_git-lfs-source/signature                          |  833 B     00:00     
Retrieving key from https://packagecloud.io/github/git-lfs/gpgkey
Importing GPG key 0xDC282033:
 Userid     : "https://packagec

In [9]:
#!pip install "transformers==4.12.3" --upgrade
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 41.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 KB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.5 MB/s eta 0:00:00:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6340e81ab1e22f083dcd4a34331e0c02c9861a2e804f086963383a28f5c5ad00
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --up

In [3]:
# Define the my final model name
mymodel_name='bertin-sts-cc-news-es'

In [4]:
import sagemaker
print(sagemaker.__version__)

# Specify your bucket name
bucket_name = 'edumunozsala-ml-sagemaker'

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
# Specify your bucket name
sagemaker_session_bucket= bucket_name
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = sess.boto_session.region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

2.101.1
sagemaker role arn: arn:aws:iam::223817798831:role/workshop-sagemaker-kfp-role
sagemaker bucket: edumunozsala-ml-sagemaker
sagemaker session region: us-east-1


In [5]:
import os
import tarfile
from sagemaker.s3 import S3Downloader

local_path = mymodel_name
# Set the S3 folder containing the model.tar.gz file
model_data= 's3://edumunozsala-ml-sagemaker/sentence-transformer-spanish/huggingface-pytorch-training-2022-08-22-17-33-29-910/output'

os.makedirs(local_path, exist_ok = True)

Download the model file from S3 to a local folder and unzip the tar.gz file

In [6]:
# download model from S3
S3Downloader.download(
    s3_uri=model_data, # s3 uri where the trained model is located
    local_path=local_path, # local path where *.tar.gz will be saved
    sagemaker_session=sess # sagemaker session used for training the model
)

In [7]:
# unzip model if it is a model.tar.gz file
tar = tarfile.open(f"{local_path}/model.tar.gz", "r:gz")
tar.extractall(path=local_path)
tar.close()
os.remove(f"{local_path}/model.tar.gz")

## Create the Sentence Transformer model 

In [10]:
from sentence_transformers import SentenceTransformer

In [12]:
local_path= local_path +'/'
print(local_path)

bertin-sts-cc-news-es/


Instantiate a SentenceTransformer model from the loca files

In [13]:
model = SentenceTransformer(local_path)

In [15]:
embeddings = model.encode('Se trata de una simple prueba del modelo generado')
print(embeddings)

[ 1.24001510e-01 -6.41292632e-01  6.75440907e-01  2.48697214e-02
 -1.39843198e-02  2.95864761e-01 -9.00002643e-02  6.84070230e-01
  1.65867675e-02  1.04857393e-01  2.14992240e-02 -1.37010172e-01
  5.51612139e-01 -7.18234405e-02 -2.77047325e-02  7.49606043e-02
 -2.68262208e-01 -3.05915207e-01 -6.03006035e-02 -3.52383077e-01
  9.73617956e-02  5.01497388e-02  2.02847958e-01 -5.72251081e-02
  1.02436110e-01 -8.14587250e-03  2.98103213e-01  8.04232776e-01
 -9.67485458e-02  4.57457989e-01  4.43778336e-01 -1.07228473e-01
  5.73192060e-01 -4.62076545e-01  1.93420649e-01  3.55567671e-02
 -3.92334014e-01  4.76867855e-01  2.19832227e-01  7.28959799e-01
  4.38820183e-01  3.68073106e-01  3.43763798e-01  9.22128707e-02
  6.78275287e-01  3.23260486e-01 -1.03598051e-02  8.82733539e-02
  3.24270517e-01  4.01368737e-01 -3.39212865e-01 -1.71711251e-01
 -1.51579633e-01  3.75644803e-01  1.48562565e-01  3.14270854e-01
  2.31097788e-01  5.26751101e-01  1.43302068e-01  4.26166534e-01
  2.00777620e-01  1.35035

Now, we test our model, we encode a pair of sentences and check the result

In [17]:
embeddings.shape

(768,)

## Upload the model to the Huggingface Hub

In [19]:
from huggingface_hub import notebook_login

notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Login successful
Your token has been saved to /home/ec2-user/.huggingface/token
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default



In [20]:
model.save_to_hub(
    "bertin-sts-cc-news-es", 
    #organization="embedding-data",
    train_datasets=["LeoCordoba/CC-NEWS-ES"],
    exist_ok=True, 
    )

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='bertin-sts-cc-news-es' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/huggingface_hub/hf_api.py:681: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/edumunozsala/bertin-sts-cc-news-es into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/476M [00:00<?, ?B/s]

To https://huggingface.co/edumunozsala/bertin-sts-cc-news-es
   7c5066c..87bc588  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/edumunozsala/bertin-sts-cc-news-es/commit/87bc58875c522a7b370731c3e77efaa131d10cf5'

And that's all! Our Sentence Transformer model is now available in the Hub.